In [1]:
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS

# Generate synthetic data
num_groups = 3
group_sizes = [5, 6, 4]
data = [torch.randn(size) for size in group_sizes]

# Define the hierarchical model
def hierarchical_model(data):
    # Prior for group-level parameters
    group_mean = pyro.sample("group_mean", dist.Normal(0, 1))
    group_std = pyro.sample("group_std", dist.Exponential(1))

    # Model for individual data points within groups
    with pyro.plate("data_plate", len(data)):
        for i, group_data in enumerate(data):
            # Prior for individual-level parameters (conditional on group-level parameters)
            mean = pyro.sample(f"mean_{i}", dist.Normal(group_mean, group_std))
            std = pyro.sample(f"std_{i}", dist.Exponential(1))
            
            # Likelihood
            pyro.sample(f"obs_{i}", dist.Normal(mean, std), obs=group_data)

# Run MCMC inference
nuts_kernel = NUTS(hierarchical_model)
mcmc = MCMC(nuts_kernel, num_samples=1000, warmup_steps=200)
mcmc.run(data)

# Get posterior samples
posterior_samples = mcmc.get_samples()

# Print posterior statistics
for param_name, samples in posterior_samples.items():
    print(f"Parameter: {param_name}")
    print(f"Mean: {torch.mean(samples, dim=0)}")
    print(f"Std: {torch.std(samples, dim=0)}")
    print()


Warmup:   0%|                                                                | 0/1200 [00:00, ?it/s]

RuntimeError: The size of tensor a (5) must match the size of tensor b (3) at non-singleton dimension 0